# House Prices Research Notebook

In [1]:
import pandas as pd 
import numpy as np

from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:

import sys
sys.path.append("drive/MyDrive/colab_notebooks/kaggle_house_prices/")
import support

In [ ]:
!pwd


/content
